In [30]:
import numpy as np
import pandas as pd

In [31]:
# Load the columns from the training dataset
def load_expected_columns(dataset_path):
    df = pd.read_csv(dataset_path)
    return list(df.columns)



# Transform a single crop_yield-style row into the full dataset format
def transform_user_input(row, expected_columns):
    columns = ['Crop', 'Crop_Year', 'Season', 'Area',
               'Annual_Rainfall', 'Fertilizer', 'Pesticide']

    if not isinstance(row, list) or len(row) != len(columns):
        raise ValueError("Row must match the expected format of crop_yield.")

    # Create DataFrame from single row
    df = pd.DataFrame([row], columns=columns)

    # Select relevant features
    features = df[['Crop_Year', 'Annual_Rainfall', 'Fertilizer', 'Pesticide', 'Crop']]

    # One-hot encode
    crop_dummies = pd.get_dummies(features['Crop'], prefix='Crop')
    # state_dummies = pd.get_dummies(features['State'], prefix='State')

    transformed = pd.concat([
        features[['Crop_Year', 'Annual_Rainfall', 'Fertilizer', 'Pesticide']],
        crop_dummies,
        # state_dummies
    ], axis=1)

    # Add any missing columns and set them to 0
    for col in expected_columns:
        if col not in transformed.columns:
            transformed[col] = 0

    # Reorder to match expected column order
    transformed = transformed[expected_columns]

    return transformed

In [45]:
import pickle
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

In [52]:
ls = load_expected_columns('dataset.csv')

In [54]:
user = ['Urad',2018,'Winter',39560,1635.9,6416632,13846]

In [55]:
x = transform_user_input(user,ls)

In [56]:
model.predict(x)

array([0.87965998])

In [38]:
x = pd.read_csv('dataset.csv')
y = pd.read_csv('dataset_y.csv')
x_test = pd.read_csv('x_test.csv')
y_test = pd.read_csv('y_test.csv')

In [39]:
model.fit(x,y)

c:\Z Local Disk (D)\Final year project\Smart-Agriculture-Support-System\Model\env\lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [42]:
model.score(x_test, y_test)

0.9536247652130968

In [43]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)